# Our Class Plan in One Big Graph

![](../images/ml.png)

# Ensemble Learning

Ensemble learning is a type of machine learning where the learning algorithm combines multiple models to improve its predictive performance. There are two main techniques within ensemble learning: bagging and boosting.

## Combining different random variables

When we combine different random variables into an ensemble, the resulting ensemble may exhibit a lower variance than each random variable. This phenomenon arises due to the correlation or covariance structure among the individual random variables.

Recall that *covariance* measures the degree to which two random variables change together. If the covariance between two variables is positive, they tend to increase or decrease together; if it's negative, they tend to move in opposite directions. On the other hand, *correlation*, which ranges from -1 to 1, measures the linear relationship between random variables where 1 indicates a perfect positive linear relationship while -1 indicates a perfect negative linear relationship, and 0 indicates no linear relationship.

When we combine different random variables where the individual random variables exhibit a low degree of correlation, or even negative covariance, the combined random variable may have reduced variability compared to the individual variables. When the individual random variables' predictions are not correlated, combining them tends to smooth out fluctuations, leading to reduced variability in the combined variable. If the individual variables' predictions are negatively correlated, combining them can cancel out some of the variability, again leading to reduced variability in the combined variable.

## An analogy with portfolio design

In finance, specifically in portfolio design theory, the goal is often to design a portfolio that achieves a desired level of return while minimizing the risk. The risk in this context is measured in terms of the variance of the returns. This is usually achieved via **portfolio diversification** where risk is reduced by combining different assets. By combining assets with uncorrelated or negatively correlated returns, investors potentially reduce the overall variance of the portfolio and lower the risk without sacrificing returns.

Diversification involves holding a variety of investments in a portfolio that have different risk-return profiles.  When assets in a portfolio have low or negative correlations with each other, their returns tend to move independently or even in opposite directions over time.  As a result, when these assets are combined in a portfolio, the overall portfolio's variance or risk is reduced due to the offsetting effects of the individual asset returns.\\

## Bagging

Bagging, also known as Bootstrap Aggregating, algorithms train multiple models independently (in most cases in parallel) on different subsets of the training data. The algorithms create these subsets by randomly sampling the training data **with replacement**.  It is important to note that each model in the ensemble is trained on a different subset.  Then the algorithm combines the predictions coming from each model through averaging (for regression models) or voting (for classification models). 

Bagging algorithms reduce the total variance by averaging over multiple noisy models, thereby improve overall prediction accuracy.

## Boosting

Boosting is an iterative ensemble method. Boosting algorithms train base learners **sequentially**. In each iteration each subsequent model focuses on the instances that the previous models failed to worked, or showed weak performance. To put another way, boosting models train each model to correct the errors made by its predecessors.  Such algorithms achieve this goal by assigning higher weights to the instances that were misclassified in the previous iterations.  In the final prediction, the result is obtained by weighted averaging where the weights are determined based on the performance of each model.

## Similarities and Differences

Now, let's delve into the differences and similarities between bagging and boosting:

1. **Training Approach**:
 + Bagging: Models are trained independently on different subsets of the training data.
 + Boosting: Models are trained sequentially, with each subsequent model focusing on the instances that previous models have struggled with.


2. **Weighting of Instances**:

 + Bagging: Each model is trained on a randomly sampled subset of the training data with replacement. All instances have equal weight.
 + Boosting: Instances are weighted based on their performance in previous iterations. Misclassified instances are given higher weight to be correctly classified in subsequent iterations.


3. **Predictor Diversity**:

 + Bagging: Models in the ensemble are typically diverse since they are trained on different subsets of data.
 + Boosting: Models in the ensemble are sequential, and each subsequent model focuses on the mistakes of the previous ones, potentially leading to less diversity.


4. **Model Combination**:

 + Bagging: Predictions are combined through averaging (for regression) or voting (for classification).
 + Boosting: Predictions are combined through weighted averaging, where the weights are determined based on the performance of each model.


5. **Performance**:

 + Bagging: Often reduces variance and can help to alleviate overfitting, particularly for unstable models.
 + Boosting: Tends to produce models with low bias and low variance, which can lead to improved performance, particularly when used with weak learners.


## Ensemble classification strategies

In the One vs. Rest (OvR) strategy (also known as one-vs-all or one-against-all) the learning algorithm trains a separate binary classifier model for each class where each classifier is responsible for distinguishing one class from the rest. In the training stage, the data points that belong to a chosen class are considered as *positive* while all other data points are considered as *negative*. In the prediction stage, each classifier produces a probability indicating whether the test point belongs to a particular class.  The final prediction is determined by the highest probability.

In the One-vs-One (OvO) strategy, also known as all-pairs or pairwise classification, the algorithm trains a binary classifier for every pair of classes. Thus for a multiclass classification problem with $k$ classes, we would need $\frac{k(k-1)}{2}$ binary classifiers. Each classifier is then trained on a subset of the training set that contains instances from only those two classes where one class is considered positive while the other is negative.  During prediction, each binary classifier produces a binary prediction indicating which of the two classes the instance belongs to.  The final predicted class is determined based on voting where each class *votes* for its class label based on the majority of votes from all binary classifiers.

OvR is typically more computationally efficient than OvO since it requires training fewer binary classifiers. However, OvO may lead to more balanced training datasets for binary classifiers, especially in cases where the number of instances per class is imbalanced.  OvR tends to perform better in practice when the number of classes is large, while OvO may be preferable when the number of classes is small or when the binary classifiers' training is not too expensive.


## Random Forest:

Random Forest is one of the most widely used bagging algorithms, particularly for classification and regression tasks. It consists of an ensemble of decision trees, where each tree is trained on a bootstrap sample of the training data and makes predictions independently.  During the training process, at each node of the tree, a random subset of features is considered for splitting, leading to decorrelated trees.  The final prediction is obtained by averaging (regression) or voting (classification) over all trees in the ensemble.

In [104]:
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

import xgboost as xgb
from lightgbm import LGBMClassifier

In [113]:
def experiment(X,y,strategy,model,test=0.25):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test)
    res = strategy(model).fit(X_train, y_train)
    y_pred = res.predict(X_test)
    return classification_report(y_test, y_pred)

### IRIS Dataset

In [114]:
iris = load_iris()
iris_X = iris.data
iris_y = iris.target

In [115]:
print(experiment(iris_X, iris_y,OneVsRestClassifier,RandomForestClassifier(n_estimators=10)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.87      0.93      0.90        14
           2       0.93      0.88      0.90        16

    accuracy                           0.92        38
   macro avg       0.93      0.93      0.93        38
weighted avg       0.92      0.92      0.92        38



In [116]:
print(experiment(iris_X, iris_y,OneVsOneClassifier,RandomForestClassifier(n_estimators=10)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.90      1.00      0.95         9
           2       1.00      0.94      0.97        18

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



In [151]:
print(experiment(iris_X, iris_y,OneVsRestClassifier,LogisticRegression(max_iter=3000)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.92      0.85      0.88        13
           2       0.87      0.93      0.90        14

    accuracy                           0.92        38
   macro avg       0.93      0.92      0.93        38
weighted avg       0.92      0.92      0.92        38



In [210]:
print(experiment(iris_X, iris_y, OneVsOneClassifier, SVC()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.93      1.00      0.96        13
           2       1.00      0.92      0.96        13

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### Digits Dataset

In [211]:
digits = load_digits()
digits_X = digits.data
digits_y = digits.target

In [236]:
print(experiment(digits_X, digits_y, OneVsRestClassifier, RandomForestClassifier(n_estimators=10)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       0.98      0.95      0.97        44
           2       1.00      1.00      1.00        49
           3       0.95      0.93      0.94        44
           4       0.98      0.90      0.94        51
           5       0.93      0.93      0.93        46
           6       0.98      1.00      0.99        43
           7       0.94      0.98      0.96        49
           8       1.00      0.89      0.94        46
           9       0.81      0.97      0.88        39

    accuracy                           0.96       450
   macro avg       0.96      0.96      0.96       450
weighted avg       0.96      0.96      0.96       450



In [247]:
print(experiment(digits_X, digits_y, OneVsRestClassifier, LogisticRegression(max_iter=4000)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        42
           1       0.94      0.98      0.96        48
           2       0.98      0.96      0.97        55
           3       0.94      0.94      0.94        50
           4       0.95      1.00      0.98        40
           5       0.98      0.95      0.96        42
           6       1.00      0.98      0.99        43
           7       0.98      1.00      0.99        40
           8       0.94      0.84      0.88        55
           9       0.87      0.97      0.92        35

    accuracy                           0.96       450
   macro avg       0.96      0.96      0.96       450
weighted avg       0.96      0.96      0.96       450



In [276]:
print(experiment(digits_X, digits_y, OneVsOneClassifier, SVC()))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        44
           1       0.98      1.00      0.99        43
           2       1.00      1.00      1.00        42
           3       1.00      1.00      1.00        49
           4       0.98      0.95      0.96        42
           5       1.00      1.00      1.00        45
           6       1.00      1.00      1.00        48
           7       1.00      1.00      1.00        40
           8       0.98      1.00      0.99        42
           9       0.98      0.98      0.98        55

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450



## ADABoost

AdaBoost is an ensemble learning algorithm that combines multiple weak models to create a stronger model. AdaBoost algorithm trains weaker models (models that perform slightly better than random guessing) iteratively on weighted training data.  It assigns higher weights to points misclassified by the previous models and thus focuses on more difficult data points in each iteration. It achieves a final prediction by combining all predictions through a weighted sum.

### Training

AdaBoost trains a sequence of weak models where each subsequent model focuses on the mistakes made by the previous ones.  At each iteration, a new model is trained on a modified version of the training data, where the weights of instances are adjusted based on their classification errors.  Even though AdaBoost can use any machine learning algorithm as its base estimator, decision trees with a single node are a common choice.  Decision stumps are simple classifiers that make decisions based on a single feature, making them computationally efficient and less prone to overfitting.

### Assigning weights to data points

At the beginning, the points in the train set are assigned equal weights. In each iteration, the algorithm simultaneously increases the weights of misclassified points while decreasing the weights for correctly classified points. This allows AdaBoost to focus more on difficult-to-classify data points. This improves the overall performance of the ensemble.

### Stopping and the final prediction

After training a predefined number of models, or until a specified stopping criterion is met, AdaBoost combines all predictions for a final prediction by taking a weighted sum of these predictions. The weights (called *boosting factors*) are determined based on the accuracy of each weak learner.  The boosting factor for each model is determined by the accuracy, with more accurate models having a higher influence on the final prediction.

AdaBoost can be adapted for both binary classification and multi-class classification tasks.  For binary classification tasks, AdaBoost typically returns class probabilities that can be transformed into binary predictions using a specified threshold.  For multi-class classification tasks, AdaBoost uses strategies like One-vs-Rest (OvR) or One-vs-One (OvO) to extend the algorithm to handle multiple classes.


In [300]:
print(experiment(iris_X,iris_y,OneVsOneClassifier,AdaBoostClassifier(estimator=LogisticRegression(max_iter=2000)),test=0.25))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00         8

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



In [309]:
print(experiment(digits_X, digits_y, OneVsOneClassifier, AdaBoostClassifier(algorithm='SAMME'),test=0.25))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        42
           1       0.92      0.98      0.95        49
           2       1.00      1.00      1.00        37
           3       0.98      0.96      0.97        45
           4       0.98      0.98      0.98        52
           5       0.93      0.95      0.94        42
           6       1.00      0.98      0.99        41
           7       0.98      1.00      0.99        42
           8       0.91      0.86      0.89        50
           9       0.96      0.98      0.97        50

    accuracy                           0.96       450
   macro avg       0.97      0.97      0.97       450
weighted avg       0.96      0.96      0.96       450



In [313]:
print(experiment(digits_X, digits_y, OneVsOneClassifier, AdaBoostClassifier(algorithm='SAMME'),test=0.25))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        52
           1       0.93      0.95      0.94        43
           2       1.00      0.95      0.98        43
           3       0.94      0.89      0.92        37
           4       1.00      0.93      0.96        57
           5       0.89      0.95      0.92        42
           6       0.97      0.97      0.97        37
           7       0.91      0.96      0.94        53
           8       0.92      0.97      0.94        35
           9       0.94      0.90      0.92        51

    accuracy                           0.95       450
   macro avg       0.95      0.95      0.95       450
weighted avg       0.95      0.95      0.95       450



In [89]:
print(experiment(digits_X, digits_y, OneVsRestClassifier, AdaBoostClassifier(estimator=SVC(max_iter=2000, probability=True), algorithm='SAMME'),test=0.25))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.00      0.00      0.00        44
           2       0.00      0.00      0.00        46
           3       0.00      0.00      0.00        57
           4       0.00      0.00      0.00        50
           5       0.10      1.00      0.18        45
           6       0.00      0.00      0.00        40
           7       0.00      0.00      0.00        42
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00        39

    accuracy                           0.10       450
   macro avg       0.01      0.10      0.02       450
weighted avg       0.01      0.10      0.02       450



/home/kaygun/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaygun/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaygun/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LightGBM

LightGBM is a gradient-boosting framework known for its efficiency, scalability, and high performance. Like any other ensemble algorithm, it iteratively combines weaker models (typically decision trees) to create a stronger model. It builds decision trees sequentially where each new tree is trained to correct the errors made by previous trees. The algorithm is widely used for supervised learning tasks such as classification and regression problems. 


A distinguishing feature of LightGBM is that it uses a histogram-based algorithm for efficient tree construction. Instead of using exact algorithms for finding the best-split points, LightGBM discretizes continuous features into discrete bins (histograms) and uses these histograms to find approximate split points. Therefore, it natively supports categorical features without the need for one-hot encoding.  Histogram-based splitting reduces memory consumption and computation time, making LightGBM highly efficient, especially on large datasets.

LightGBM grows trees leaf-wise, instead of the traditional level-wise approach used in other boosting algorithms. In this approach, the algorithm grows the tree by splitting the leaf giving the largest decrease in the loss function. This leads to deeper trees with fewer nodes, allowing the algorithm to capture complex patterns more efficiently.

LightGBM uses gradient-based optimization techniques to train each tree in the ensemble.  It minimizes a differentiable loss function by iteratively updating the tree structure to move in the direction of the negative gradient of the loss function. This gradient-based approach enables LightGBM to efficiently optimize complex objective functions, including custom loss functions.


LightGBM is designed for parallel and distributed computing, allowing it to scale efficiently to large datasets and distributed computing environments. It supports multi-threading for parallel training on multi-core CPUs and distributed training on distributed computing frameworks such as Apache Spark. Due to its efficient algorithms and optimization techniques, LightGBM is known for its high performance and scalability. It is often the algorithm of choice for large-scale machine learning tasks, such as those involving high-dimensional datasets or massive amounts of data. 


LightGBM provides various regularization techniques to prevent overfitting and improve generalization performance.  Regularization parameters, such as `max_depth`, `min_child_samples`, and `lambda` (L2 regularization), can be tuned to control the complexity of the trees and reduce overfitting.

For further information, read the [documentation](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html).

In [331]:
print(experiment(digits_X, digits_y, OneVsRestClassifier, LGBMClassifier(num_leaves=10, n_estimators=50),test=0.25))

[LightGBM] [Info] Number of positive: 132, number of negative: 1215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 834
[LightGBM] [Info] Number of data points in the train set: 1347, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097996 -> initscore=-2.219697
[LightGBM] [Info] Start training from score -2.219697
[LightGBM] [Info] Number of positive: 130, number of negative: 1217
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 834
[LightGBM] [Info] Number of data points in the train set: 1347, number of used features: 53
[LightGBM] [Info] [binary:Bo

## XGBoost

XGBoost, short for Extreme Gradient Boosting, belongs to the family of gradient boosting algorithms.  XGBoost is a powerful and efficient gradient-boosting framework widely used for supervised learning tasks, particularly for classification, regression, and ranking problems. XGBoost employs a tree-boosting algorithm, where each tree is added to the ensemble sequentially to correct the errors made by the existing ensemble. XGBoost also uses techniques such as tree pruning and split finding to control the complexity of the trees and improve computational efficiency. It starts with an initial prediction (often the mean value for regression or the class with the highest frequency for classification) and iteratively adds trees to minimize the loss function. It uses an efficient algorithm to find the optimal split points for each feature, considering the information gain and the regularization term. 

XGBoost's regularized objective function consists of two main components: a loss function and a regularization term. The loss function measures the difference between the predicted and actual values, while the regularization term penalizes the complexity of the model to prevent overfitting. It minimizes the loss function by iteratively updating the model parameters (e.g., tree structure, leaf weights) in the direction of the negative gradient of the loss function. Common loss functions include squared loss for regression and softmax for multiclass classification.  Regularization parameters, such as `max_depth`, `min_child_weight`, and `gamma`, can be tuned to control the complexity of the trees and the regularization strength.

XGBoost can handle missing values in the dataset by learning the optimal direction to handle missing values during training, thus eliminating the need for imputation or preprocessing steps.  XGBoost is also designed for parallel and distributed computing, allowing it to scale efficiently to large datasets and distributed computing environments. It supports multi-threading for parallel training on multi-core CPUs and distributed training on distributed computing frameworks.  It often outperforms other gradient-boosting frameworks and is widely used in machine-learning competitions and real-world applications.


In [111]:
def experimentXG(X,y,num_classes,rounds=50,test=0.25):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    params = {
        'objective': 'multi:softmax',  # Multiclass classification objective
        'num_class': num_classes,  # Number of classes in the dataset
        'eval_metric': 'merror'  # Evaluation metric: multiclass classification error rate
    }
    model = xgb.train(params, dtrain, rounds)
    y_pred = model.predict(dtest)
    
    return classification_report(y_test, y_pred)

In [353]:
print(experimentXG(digits_X,digits_y,10))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97        44
           1       1.00      0.94      0.97        47
           2       1.00      0.94      0.97        52
           3       0.93      0.93      0.93        44
           4       0.92      0.92      0.92        48
           5       0.90      0.95      0.93        40
           6       0.98      0.98      0.98        42
           7       0.94      0.98      0.96        50
           8       0.95      0.98      0.96        42
           9       0.93      0.93      0.93        41

    accuracy                           0.95       450
   macro avg       0.95      0.95      0.95       450
weighted avg       0.95      0.95      0.95       450



## Optimizers

Assume we have a function $f\colon \Omega\to \mathbb{R}$ where $\Omega$ is a compact (closed and bounded) set in $\mathbb{R}^n$. How do we find its extremum (max or min) points? 

Remember that most machine learning algorithms are designed to optimize a loss, a reward, or a penalty function. Thus optimizers play a crucial role in training machine learning models by adjusting the model parameters (e.g., weights and biases) to minimize a predefined loss function. 

Here are some common optimizer choices:

### Gradient Descent

Gradient Descent is the most fundamental optimization algorithm. It iteratively moves in the direction of the negative gradient of the loss function.

$$ x_{n+1} = x_n - \epsilon (\nabla f)(x_n) $$

Variants include:
- **Stochastic Gradient Descent (SGD)**: Updates parameters using the gradient of the loss computed on a single randomly chosen training instance. It's computationally efficient but may have high variance in parameter updates.
- **Mini-batch Gradient Descent**: Updates parameters using the gradient of the loss computed on a small batch of training instances. It strikes a balance between the efficiency of SGD and the stability of full-batch Gradient Descent.

### RMSprop (Root Mean Square Propagation)

RMSprop is an adaptive optimization algorithm that adjusts the learning rates for each parameter based on the magnitude of recent gradients.  It divides the learning rate by a running average of the squared gradients, which helps to prevent the learning rate from decaying too quickly or too slowly. 

### Adam (Adaptive Moment Estimation)

Adam is an adaptive optimization algorithm that combines the benefits of both momentum and RMSprop. It maintains per-parameter learning rates based on estimates of the first and second moments of the gradients. Adam is widely used in deep learning and is known for its fast convergence and robustness to noisy gradients.

### Adagrad (Adaptive Gradient Algorithm)

Adagrad is an adaptive optimization algorithm that adjusts the learning rates for each parameter that decreases over time as the sum of squared gradients accumulates. Adagrad is suitable for sparse data or problems with large differences in feature scales but may suffer from a diminishing learning rate problem.

### AdaDelta

AdaDelta is an extension of Adagrad that addresses the diminishing learning rate problem by using a decaying average of the learning rates based on a moving window of the past gradients without explicitly maintaining learning rates. 

### AdamW

AdamW is a variant of Adam that introduces weight decay regularization to stabilize training and prevent overfitting.  It decouples weight decay from the optimization step, leading to more stable training dynamics and improved generalization performance. AdamW is particularly effective in training deep neural networks and has become popular in the deep learning community.

### LBFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno)

LBFGS is a quasi-Newton optimization algorithm that approximates the Hessian matrix to perform line search optimization.  It maintains an estimate of the inverse Hessian matrix using limited memory, making it memory-efficient and suitable for problems with large numbers of parameters. LBFGS is often used in optimization problems with smooth and convex objective functions.


In [ ]:
print(experiment(digits_X, digits_y, OneVsRestClassifier, LGBMClassifier(num_leaves=10, n_estimators=50),test=0.25))

## Yoklama

- 090190328
- 090200304
- 090190315
- 090200358
- 090210362
- 090210361

